In [1]:
import pandas as pd
import numpy as np 
from scipy.stats import skew
import klib

In [2]:
df = pd.read_csv(r"D:\2025 lessons\AI+ML course amaliyot\Datasets\housing_in_london.csv")
df.info()                       # * target varialbe is average_price

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           13549 non-null  object 
 1   area           13549 non-null  object 
 2   average_price  13549 non-null  int64  
 3   code           13549 non-null  object 
 4   houses_sold    13455 non-null  float64
 5   no_of_crimes   7439 non-null   float64
 6   borough_flag   13549 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 741.1+ KB


In [3]:
df = klib.data_cleaning(df) 
df.info()

Shape of cleaned data: (13549, 7) - Remaining NAs: 6204


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.49 MB (-68.06%)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   date           13549 non-null  category
 1   area           13549 non-null  category
 2   average_price  13549 non-null  int32   
 3   code           13549 non-null  category
 4   houses_sold    13455 non-null  float32 
 5   no_of_crimes   7439 non-null   float32 
 6   borough_flag   13549 non-null  int8    
dtypes: category(3), float32(2), int32(1), int8(1)
memory usage: 238.3 KB


In [4]:
for col in df.columns[df.isnull().any()]:     # ?  missing value  numeric type features ichida bulganligi sababli 
    df[col] = df[col].fillna(df[col].mean())    # ! fill missing values

df.isnull().sum()

date             0
area             0
average_price    0
code             0
houses_sold      0
no_of_crimes     0
borough_flag     0
dtype: int64

# Encoding

In [5]:
from sklearn.preprocessing import LabelEncoder

df.drop(columns = ['code'], inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   date           13549 non-null  category
 1   area           13549 non-null  category
 2   average_price  13549 non-null  int32   
 3   houses_sold    13549 non-null  float32 
 4   no_of_crimes   13549 non-null  float32 
 5   borough_flag   13549 non-null  int8    
dtypes: category(2), float32(2), int32(1), int8(1)
memory usage: 223.7 KB


In [6]:
category_col= df.select_dtypes(include='category').columns
for col in category_col:
    df[col] = LabelEncoder().fit_transform(df[col])
df.head()

,date,area,average_price,houses_sold,no_of_crimes,borough_flag
0,0,6,91449,17.0,2158.352051,1
1,1,6,82203,7.0,2158.352051,1
2,2,6,79121,14.0,2158.352051,1
3,3,6,77101,7.0,2158.352051,1
4,4,6,84409,10.0,2158.352051,1


# Without scaling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error

model = RandomForestRegressor(n_estimators=100,random_state=42)

In [33]:
# df_noscale = df.copy(deep=True)

y_ns = df['average_price'] #   No scaling
x_ns = df.drop(columns=['average_price'])





In [37]:
x_train_ns ,x_temp_ns,y_train_ns,y_temp_ns=train_test_split(x_ns,y_ns,test_size=0.2,random_state=42)
x_test_ns,x_val_ns,y_test_ns,y_val_ns=train_test_split(x_temp_ns,y_temp_ns,test_size=0.5,random_state=42)

noscale_model = model.fit(x_train_ns,y_train_ns)
noscale_model


RandomForestRegressor(random_state=42)

In [38]:
y_pred_ns=noscale_model.predict(x_test_ns)
mse_noscale=mean_squared_error(y_test_ns,y_pred_ns)
r2_noscale=r2_score(y_test_ns,y_pred_ns)



# With Scaling

In [39]:
from sklearn.preprocessing import StandardScaler
df.head()

,date,area,average_price,houses_sold,no_of_crimes,borough_flag
0,0,6,91449,17.0,2158.352051,1
1,1,6,82203,7.0,2158.352051,1
2,2,6,79121,14.0,2158.352051,1
3,3,6,77101,7.0,2158.352051,1
4,4,6,84409,10.0,2158.352051,1


In [43]:
df['average_price'] = pd.DataFrame(StandardScaler().fit_transform(df[['average_price']]))
df['average_price'].head()

0   -0.917169
1   -0.966452
2   -0.982880
3   -0.993647
4   -0.954694
Name: average_price, dtype: float64

In [44]:
df['no_of_crimes']  = pd.DataFrame(StandardScaler().fit_transform(df[['no_of_crimes']]))
df['no_of_crimes'].head()

0   -1.040631e-08
1   -1.040631e-08
2   -1.040631e-08
3   -1.040631e-08
4   -1.040631e-08
Name: no_of_crimes, dtype: float32

In [46]:
y_scaled = df['average_price']
x_scaled = df.drop(columns='average_price')
x_scaled

,date,area,houses_sold,no_of_crimes,borough_flag
0,0,6,17.000000,-1.040631e-08,1
1,1,6,7.000000,-1.040631e-08,1
2,2,6,14.000000,-1.040631e-08,1
3,3,6,7.000000,-1.040631e-08,1
4,4,6,10.000000,-1.040631e-08,1
...,...,...,...,...,...
13544,296,12,64605.000000,-1.040631e-08,0
13545,297,12,68677.000000,-1.040631e-08,0
13546,298,12,67814.000000,-1.040631e-08,0
13547,299,12,3893.994141,-1.040631e-08,0


In [26]:
mse_noscale


422029712.3357187